# Deploy LLM as API

So, now you can easily make chatbots or conversational bots using both open source OLLAMA LLMs and OPENAI LLMs, so we made these chatbots using chains where we have to first make a prompt template, then load llm and then get the output as a string but in real world scenarios we would like to easily integrate these chatbots in different platforms like apps and websites rather than creating chains for each of them separately, which can be easily done using **APIs** In this tutorial we will learn how can we makes APIs of these chains using the amazing langchain ecosystem where we are provided **Langserve** which will help us to create APIs using **FastAPI** which will further allow us to easily deployment purposes

## Langserve : It helps developers deploy Langchain runnables and chains as REST API. This library isi integrated with FASTAPI and uses PYNDATIC for data validation

### Required Libraries : requirements.txt

> 1) langchain_openai
> 2) langchain_core
> 3) python-dotenv
> 4) streamlit
> 5) langchain_community
> 6) langserve
> 7) fastapi
> 8) uvicorn
> 9) sse_starlette

install all these libraries in your virtual environment using one of the techniquies discussed in **Chapter:2**

### app.py

```python

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from fastapi import FastAPI
from dotenv import load_dotenv
import os
from langserve import add_routes
import uvicorn

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

app = FastAPI(
    title="Langchain Server",
    description="A simple API server",
    version="1.0.0"
)

add_routes(
    app,
    ChatOpenAI(),
    path='/openai'
)

model = ChatOpenAI()

#Ollama llm

llm = Ollama(model="llama2")

#chat prompt template
prompt1 = ChatPromptTemplate.from_messages(
    [
        ("Give me 5 synonyms of {word}")
    ]
)

prompt2 = ChatPromptTemplate.from_messages(
    [
        ("Give me best five uses of {word}")
    ]
)

add_routes(
    app,
    prompt1 | model,
    path = "/synonyms"
)

add_routes(
    app,
    prompt2 | llm,
    path = "/uses"
)

if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)


after setting up app.py run app.py by `python app.py`

Now go to localhost and check if server is running and then go to `http://localhost:8000/docs` you will see all the defined routes in your langchain server

Now our next step is how we can easily use or access these apis

### client.py 

Now we will learn how can we use the api created by us in different platforms like websites so we are going to create a simple website using streamlit and use the above api created to generate some content in this website using the api routes /synonyms and /uses | Here we will create a streamlit application and call these APIs


```python

import streamlit as st
import requests

def openai_response(input_text):
    response = requests.post("http://localhost:8000/synonyms/invoke", 
                        json={'input':{'word': input_text }})
    return response.josn()['output']['content']

def ollama_response(input_text):
    response = requests.post("http://localhost:8000/uses/invoke", 
                        json={'input':{'word': input_text }})
    return response.josn()['output']
                            


input_text1 = st.input_text("Write the word whose synonym you want to know")
input_text2 = st.input_text("Write the word whose uses you want to know")

if input_text1:
    response = openai_response(input_text1)
    st.write(response)

if input_text2:
    response = ollama_response(input_text2)
    st.write(response)

Now in the above code the client is interacting with the API